In [11]:
import pandas as pd
import numpy as np
from googletrans import Translator

In [7]:
df = pd.read_csv("../../delphes/data/final2_clean.csv", index_col=0)

In [8]:
df.head()

,mep_id,name,country,group,nat_group,twitter,content
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,"['W tym dniu, w tym miejscu, w tej godzinie pr..."
1,189525,Asim ADEMOV,Bulgaria,Group of the European People's Party (Christia...,Citizens for European Development of Bulgaria,AdemovAsim,['RT @ECinBulgaria: 📢 Остана 1⃣ седмица! Преди...
2,124831,Isabella ADINOLFI,Italy,Non-attached Members,Movimento 5 Stelle,Isa_Adinolfi,"[""Sembra un film, ma purtroppo è realtà: le im..."
6,125045,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,Partido Socialista Obrero Español,ClaraAguilera7,['RT @ClaraAguilera7: Debate e importantes vot...
7,204335,Alviina ALAMETSÄ,Finland,Group of the Greens/European Free Alliance,Vihreä liitto,alviinaalametsa,['Toimeentulotukea korotetaan 75e koronakriisi...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 0 to 704
Data columns (total 7 columns):
mep_id       474 non-null int64
name         474 non-null object
country      474 non-null object
group        474 non-null object
nat_group    474 non-null object
twitter      474 non-null object
content      474 non-null object
dtypes: int64(1), object(6)
memory usage: 29.6+ KB


In [26]:
sample = df.content.map(lambda x: x.split(","))

In [12]:
translator = Translator()

In [38]:
def trad(X):
    res = []
    for i in X:
        res.append(translator.translate(i, dest='en').text)
    return res

In [34]:
trad_content = sample[:2].map(lambda x: trad(x))

In [37]:
translator.translate("w tej godzinie proszę o jedno: nie o puste slogany a o zrozumienie", dest='en').text

'at this hour I am asking for one thing: not for empty slogans, but for understanding'

In [40]:
trad(sample[0])

["['W tym dniu",
 'in this place',
 'at this hour I am asking for one thing: not for empty slogans but for understanding',
 "how each starts ... https://t.co/7h6UV2vEtw '",
 "'RT @Pontifex_en: God doesn't need to be protected by anyone and doesn't want to",
 " aby Jego imię było używane do terroryzowania ludzi. Proszę wszyst…'",
 "'I submitted applications to the Supreme Administrative Court to speed up the examination of my cassation complaints and to the Director of the Chamber of Administration S ... https://t.co/Q3gecVvXke'",
 "'If only empathy",
 'goodness and conciliation had a human face',
 ' byłaby to twarz Twoja',
 " Henryku. Spoczywaj w pokoju. \\nHenryk Wujec 1940-2020.'",
 "'Despite this",
 "that I do not agree with the tax authority's interpretation",
 'I inform',
 " iż wszystkie podatki tuż po decyzji… https://t.co/wkupTyjnGU'",
 "'It doesn't surprise me at all",
 'that since yesterday the contents of the charges have been known to the government media and asked questions